# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

CustomerID FirstName   LastName  SalesID  ProductID           ProductName  \
0       61288      Rosa   Andersen   134196        229  Bread - Hot Dog Buns   
1       77352     Myron     Murray  6167892        229  Bread - Hot Dog Buns   
2       40094     Susan  Stevenson  5970885        229  Bread - Hot Dog Buns   
3       23548    Tricia    Vincent  6426954        229  Bread - Hot Dog Buns   
4       78981     Scott      Burch   819094        229  Bread - Hot Dog Buns   

   Quantity  
0        16  
1        20  
2        11  
3         6  
4        20

## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
quantity_customer = data.groupby(['CustomerID', 'ProductName']).agg({'Quantity':'sum'}).sort_values(['Quantity'], ascending=False)
quantity_customer.head()

Quantity
CustomerID ProductName                               
90069      Longos - Grilled Salmon With Bbq        92
80694      Yeast Dry - Fermipan                    84
96615      Veal - Eye Of Round                     75
97063      Bread - French Baquette                 75
95017      Coffee - Irish Cream                    75

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
customers = data.pivot_table(index='ProductName', columns='CustomerID', values='Quantity').fillna(0)
customers.head()

CustomerID                         33     200    264    356    412    464    \
ProductName                                                                   
Anchovy Paste - 56 G Tube            0.0    0.0    0.0    0.0    0.0    0.0   
Appetizer - Mini Egg Roll, Shrimp    0.0    0.0    0.0    0.0    0.0    0.0   
Appetizer - Mushroom Tart            0.0    0.0    0.0    0.0    0.0    0.0   
Appetizer - Sausage Rolls            0.0    0.0    0.0    0.0    0.0    0.0   
Apricots - Dried                     1.0    0.0    0.0    0.0    1.0    0.0   

CustomerID                         477    639    649    669    ...  97697  \
ProductName                                                    ...          
Anchovy Paste - 56 G Tube            0.0    1.0    0.0    0.0  ...    0.0   
Appetizer - Mini Egg Roll, Shrimp    0.0    0.0    0.0    0.0  ...   25.0   
Appetizer - Mushroom Tart            0.0    1.0    0.0    0.0  ...   25.0   
Appetizer - Sausage Rolls            0.0    0.0    0.0    0.0  ...    0.0   
Apricots - Dried                     0.0    0.0    0.0    0.0  ...    0.0   

CustomerID                         97753  97769  97793  97900  97928  98069  \
ProductName                                                                   
Anchovy Paste - 56 G Tube           25.0    0.0    0.0    0.0    0.0    0.0   
Appetizer - Mini Egg Roll, Shrimp   25.0    0.0    0.0    0.0    0.0    0.0   
Appetizer - Mushroom Tart            0.0    0.0    0.0    0.0    0.0    0.0   
Appetizer - Sausage Rolls            0.0    0.0    0.0   25.0   25.0   25.0   
Apricots - Dried                    25.0    0.0    0.0    0.0    0.0    0.0   

CustomerID                         98159  98185  98200  
ProductName                                             
Anchovy Paste - 56 G Tube            0.0    0.0    0.0  
Appetizer - Mini Egg Roll, Shrimp    0.0    0.0    0.0  
Appetizer - Mushroom Tart            0.0   25.0    0.0  
Appetizer - Sausage Rolls            0.0   25.0    0.0  
Apricots - Dried                     0.0    0.0    0.0  

[5 rows x 1000 columns]

## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [6]:
sim_matrix = squareform(pdist(customers.T, 'euclidean'))
distances = pd.DataFrame(1/(1+sim_matrix), index = customers.columns, columns = customers.columns)
distances.head()

CustomerID     33        200       264       356       412       464    \
CustomerID                                                               
33          1.000000  0.085297  0.093953  0.091747  0.087410  0.089695   
200         0.085297  1.000000  0.085638  0.085297  0.080070  0.083020   
264         0.093953  0.085638  1.000000  0.088152  0.089301  0.087047   
356         0.091747  0.085297  0.088152  1.000000  0.085983  0.086688   
412         0.087410  0.080070  0.089301  0.085983  1.000000  0.085638   

CustomerID     477       639       649       669    ...     97697     97753  \
CustomerID                                          ...                       
33          0.085297  0.088913  0.088152  0.089695  ...  0.004809  0.005108   
200         0.084959  0.083651  0.085638  0.087047  ...  0.004825  0.005121   
264         0.085638  0.086333  0.087047  0.087047  ...  0.004822  0.005115   
356         0.085983  0.091325  0.085983  0.087410  ...  0.004814  0.005111   
412         0.085638  0.089301  0.084959  0.087779  ...  0.004808  0.005131   

CustomerID     97769     97793     97900     97928     98069     98159  \
CustomerID                                                               
33          0.004996  0.005421  0.004920  0.005023  0.004880  0.005026   
200         0.005014  0.005448  0.004925  0.005032  0.004909  0.005042   
264         0.004996  0.005441  0.004932  0.005055  0.004894  0.005042   
356         0.004999  0.005437  0.004920  0.005036  0.004871  0.005042   
412         0.004996  0.005441  0.004925  0.005042  0.004876  0.005039   

CustomerID     98185     98200  
CustomerID                      
33          0.004549  0.004883  
200         0.004553  0.004879  
264         0.004566  0.004883  
356         0.004563  0.004886  
412         0.004568  0.004903  

[5 rows x 1000 columns]

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [7]:
recomended = distances[90069].sort_values(ascending= False)
recomended.head(6)

CustomerID
90069    1.000000
28030    0.005410
14012    0.005343
23001    0.005331
13338    0.005330
21734    0.005329
Name: 90069, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [8]:
top_five_recomendations = list(recomended[1:6].index)
top_five_recomendations.sort()
top_five_customers = quantity_customer.loc[top_five_recomendations]

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [9]:
similar_products = quantity_customer.groupby(['ProductName']).agg({'Quantity':'sum'})
similar_products.sort_values(by='Quantity', ascending=False).head()

Quantity
ProductName                       
Cookies - Assorted            2403
Flavouring - Orange           2397
Fenngreek Seed                2391
Wine - White Cab Sauv.on      2387
Bandage - Flexible Neon       2347

## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [10]:
purchased = pd.merge(similar_products,customers[90069], on = 'ProductName')
purchased['NotBought'] = purchased.Quantity - purchased[90069]
purchased.sort_values(by='NotBought', ascending=False).head(7)

Quantity  90069  NotBought
ProductName                                                 
Cookies - Assorted                    2403    0.0     2403.0
Flavouring - Orange                   2397    0.0     2397.0
Fenngreek Seed                        2391    0.0     2391.0
Wine - White Cab Sauv.on              2387   23.0     2364.0
Bandage - Flexible Neon               2347    0.0     2347.0
Oil - Shortening - All - Purpose      2344    0.0     2344.0
Beef - Montreal Smoked Brisket        2344    0.0     2344.0

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [11]:
recommendations = {}
total_customers = data['CustomerID'].unique()
total_customers.sort()

def suggest_products_for_customer(cust):
    
    similar_customers = list(distances[cust].sort_values(ascending= False)[1:6].index)
    similar_records = quantity_customer.loc[top_five_recomendations].groupby(['ProductName']).agg({'Quantity':'sum'})
    purchased = pd.merge(similar_records,customers[cust], on = 'ProductName')
    purchased['NotBought'] = purchased.Quantity - purchased[cust]
    
    return list(purchased.sort_values(by='NotBought', ascending=False).index[0:5])

for i in total_customers:
    items = suggest_products_for_customer(i)
    recommendations.update({i:items})

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [12]:
df = pd.DataFrame(recommendations).T.rename(columns={0:'Item1',1:'Item2',2:'Item3', 3:'Item4',4:'Item5'})
df.head()

Item1                         Item2  \
33   Pastry - Cheese Baked Scones    Bread - Raisin Walnut Oval   
200  Pastry - Cheese Baked Scones    Bread - Raisin Walnut Oval   
264  Pastry - Cheese Baked Scones    Bread - Raisin Walnut Oval   
356  Pastry - Cheese Baked Scones  Chocolate - Compound Coating   
412  Pastry - Cheese Baked Scones    Bread - Raisin Walnut Oval   

                            Item3                         Item4  \
33   Chocolate - Compound Coating          Cinnamon Buns Sticky   
200  Chocolate - Compound Coating          Cinnamon Buns Sticky   
264  Chocolate - Compound Coating          Cinnamon Buns Sticky   
356    Bread - Raisin Walnut Oval          Cinnamon Buns Sticky   
412          Cinnamon Buns Sticky  Chocolate - Compound Coating   

                            Item5  
33    Oil - Shortening,liqud, Fry  
200   Oil - Shortening,liqud, Fry  
264  Wine - Charddonnay Errazuriz  
356   Oil - Shortening,liqud, Fry  
412   Oil - Shortening,liqud, Fry

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [13]:
matrix = squareform(pdist(customers.T, 'cosine'))
distances = pd.DataFrame(1/(1+matrix), index = customers.columns, columns = customers.columns)
    
for i in total_customers:
    items = suggest_products_for_customer(i)
    recommendations.update({i:items})

df = pd.DataFrame(recommendations).T.rename(columns={0:'Item1',1:'Item2',2:'Item3', 3:'Item4',4:'Item5'})
df.head()

Item1                         Item2  \
33   Pastry - Cheese Baked Scones    Bread - Raisin Walnut Oval   
200  Pastry - Cheese Baked Scones    Bread - Raisin Walnut Oval   
264  Pastry - Cheese Baked Scones    Bread - Raisin Walnut Oval   
356  Pastry - Cheese Baked Scones  Chocolate - Compound Coating   
412  Pastry - Cheese Baked Scones    Bread - Raisin Walnut Oval   

                            Item3                         Item4  \
33   Chocolate - Compound Coating          Cinnamon Buns Sticky   
200  Chocolate - Compound Coating          Cinnamon Buns Sticky   
264  Chocolate - Compound Coating          Cinnamon Buns Sticky   
356    Bread - Raisin Walnut Oval          Cinnamon Buns Sticky   
412          Cinnamon Buns Sticky  Chocolate - Compound Coating   

                            Item5  
33    Oil - Shortening,liqud, Fry  
200   Oil - Shortening,liqud, Fry  
264  Wine - Charddonnay Errazuriz  
356   Oil - Shortening,liqud, Fry  
412   Oil - Shortening,liqud, Fry